In [ ]:
"""
We will train an attention seq2seq for translating from Nahuatl to Spanish, using the Pytorch tutorial: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
"""

'\nWe will train an attention seq2seq for translating from Nahuatl to Spanish, using the Pytorch tutorial: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html\n'

In [ ]:
!pip install fasttext

In [ ]:
from __future__ import unicode_literals, print_function, division
import time
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import math
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import pandas as pd

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

import fasttext
import fasttext.util
# from gensim.models.fasttext import FastTextKeyedVectors


In [ ]:
fasttext.util.download_model('nah', if_exists='ignore')
nh_ft = fasttext.load_model('cc.nah.300.bin')
nh_ft.get_dimension()
nh_ft.get_nearest_neighbors('coatl')

[(0.8674371242523193, 'cuechcoatl'),
 (0.8571669459342957, 'Olcoatl'),
 (0.8520733118057251, 'Itzcoatl'),
 (0.8365020155906677, 'Zolcoatl'),
 (0.8254373669624329, 'Cuechcoatl'),
 (0.820296585559845, 'Chiauhcoatl'),
 (0.8056610822677612, 'Micoatl'),
 (0.7976199984550476, 'Ocelocoatl'),
 (0.7928087115287781, 'Roquercoatl'),
 (0.7875798344612122, 'Coatl')]

In [ ]:
nah_vectors = torch.from_numpy(nh_ft.get_input_matrix())
print(nah_vectors.size())

torch.Size([2116961, 300])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
SOS_token = 0
EOS_token = 1
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def split_corpus(path):
  #Split method thanks to https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
  #returns a 60% - 20% -20% training-validation-test split
  df = pd.read_csv(path, delimiter="|")
  return np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

def readLangs(lang1, lang2,corpus, reverse=False):
    print("Reading lines...")

    # lang_df = pd.read_csv("parallel_nh-es.csv", delimiter="|")
    # print(lang_df.head())
    # print(lang_df.columns.to_list())
    pairs = corpus[['Nahuatl', 'Spanish']].to_numpy()
    # Normalizes words
    pairs = [[normalizeString(y) for y in x] for x in pairs]
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
# May be incresed to get more samples
MAX_LENGTH = 20

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
    # and p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, corpus,reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2,corpus,reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

training,validation,test = split_corpus("parallel_nh-es.csv")
input_lang, output_lang, pairs = prepareData('nh', 'es',training, True)
print(random.choice(pairs))


Reading lines...
Read 4692 sentence pairs
Trimmed to 1441 sentence pairs
Counting words...
Counted words:
es 3276
nh 3731
['y lo ponian a los pies de los apostoles . y era repartido a cada uno segun tenia necesidad', 'niman quinmactiliayaj on apostoles . niman on apostoles quinxelohuiliayaj ocsequimej quen se quipolojticaj .']


In [ ]:
"""Set up the encoder and decoder RNN models!"""


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,word_embeds=None):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        if word_embeds is not None:
            self.embedding.weight = nn.Parameter(
                word_embeds)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)



In [ ]:

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [ ]:
"""Prepare the training data."""


def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [ ]:
"""
Model training code
"""

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH,uses_attn=False):
    encoder_hidden = encoder.initHidden()
    # print("uses attn", uses_attn)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(
        max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if uses_attn:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
              loss += criterion(decoder_output, target_tensor[di])
              decoder_input = target_tensor[di]  # Teacher forcing
            # TODO: remove encoder_outputs parameter if not using attention
            # TODO: remove decoder_attention from output if not using attention
            else:
              decoder_output, decoder_hidden = decoder(
                  decoder_input, decoder_hidden)
              loss += criterion(decoder_output, target_tensor[di])
              decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            if uses_attn:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
              topv, topi = decoder_output.topk(1)
              decoder_input = topi.squeeze().detach()  # detach from history as input
            # TODO: remove encoder_outputs parameter if not using attention
            # TODO: remove decoder_attention from output if not using attention
            else:
              decoder_output, decoder_hidden = decoder(
                  decoder_input, decoder_hidden)
              topv, topi = decoder_output.topk(1)
              decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [ ]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))



In [ ]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01,attn=False):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion,uses_attn=attn)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [ ]:
"""
Evaluation"""


def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(
            max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            try:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
              decoder_attentions[di] = decoder_attention.data
            except:
              decoder_output, decoder_hidden = decoder(
                  decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]



In [ ]:
def evaluateRandomly(encoder, decoder,pairs, n=1):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


"""Calculating the BLEU score on a subset of the test data"""


'Calculating the BLEU score on a subset of the test data'

In [ ]:
def evaluate_bleu(encoder, decoder,pairs, n=10):
    """To speed up testing, we only evaluate BLEU score on n test sentences."""
    references = []
    predictions = []
    for pair in pairs[:n]:
        references.append(pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0])
        predictions.append(output_words)
    #smoothing function reference here: https://www.nltk.org/_modules/nltk/translate/bleu_score.html
    score = corpus_bleu(references, predictions,smoothing_function=SmoothingFunction().method3)
    print('BLEU score:', score)


In [ ]:

"""Finally, we can actually run and test the model (without attention)!"""
# TODO test this once everything else works
hidden_size = 300
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# TODO: try DecoderRNN without attention (remove dropout_p parameter)
decoder1 = DecoderRNN(hidden_size,output_lang.n_words).to(device)
attn_decoder1 = AttnDecoderRNN(
    hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# TO speed up training, you can reduce 75000 to 5000
trainIters(encoder1, decoder1, 5000, print_every=5000)
trainIters(encoder2, attn_decoder1, 5000, print_every=5000,attn=True)




12m 29s (- 0m 0s) (5000 100%) 4.8099
14m 7s (- 0m 0s) (5000 100%) 4.7873


In [ ]:
_,_,val_pairs = prepareData('nh', 'es',validation, True)
_,_,test_pairs = prepareData('nh', 'es',test, True)

Reading lines...
Read 1564 sentence pairs
Trimmed to 510 sentence pairs
Counting words...
Counted words:
es 1692
nh 1832
Reading lines...
Read 1565 sentence pairs
Trimmed to 465 sentence pairs
Counting words...
Counted words:
es 1549
nh 1720


In [ ]:
# evaluateRandomly(encoder1, attn_decoder1,val_pairs)
evaluate_bleu(encoder1, decoder1,val_pairs,n=len(val_pairs))
evaluate_bleu(encoder2,attn_decoder1,val_pairs,n=len(val_pairs))
evaluate_bleu(encoder1,attn_decoder1,val_pairs,n=len(val_pairs))
evaluate_bleu(encoder2,decoder1,val_pairs,n=len(val_pairs))

BLEU score: 0.0006377897438035261


In [ ]:
def save_translator(encoder,decoder,path1="encoder",path2="decoder"):
  torch.save(encoder,path1)
  torch.save(decoder,path2)

def load_translator(encoder_path,decoder_path):
  return torch.load(encoder_path) , torch.load(decoder_path)

In [ ]:
def evaluate_saved(encoder_path,decoder_path,pairs):
  evaluate_bleu(*load_translator(encoder_path,decoder_path),pairs)

In [ ]:
# save_translator(encoder1,attn_decoder1)
# save_translator(encoder1,attn_decoder1,path1="/content/drive/MyDrive/NLP_project/encoder1",path2="/content/drive/MyDrive/NLP_project/decoder1")
evaluate_saved("encoder","decoder",val_pairs)


BLEU score: 0.010107123741942224
